# Import Packages

In [2]:
# YOLO Model
from ultralytics import YOLO

# Utils
import glob
import pandas as pd
import numpy as np

# Train Model

In [ ]:
# Train YOLO Model
model = YOLO(r"C:\Users\Irsyadobby\Documents\BDC 2023\Model\yolov5mu.pt")

result = model.train(data = r"C:\Users\Irsyadobby\Documents\BDC 2023\V5 BDC.yaml", imgsz = 640, epochs = 100, lr0 = 0.001, dropout = 0.1, optimizer = "AdamW")


# Predict Test Dataset
model.predict(source = r"C:\Users\Irsyadobby\Documents\BDC 2023\Penyisihan\Data Test for BDC 2023 - Penyisihan\Data Test for BDC 2023 - Penyisihan", save = True, save_txt = True)

# Post-Processing

In [5]:
def data_label(data) :
    for row in range(0, len(data)) :
        if 0 <= data.iloc[row,0] <= 25:
            data.loc[row, 'label'] = chr(data.iloc[row ,0] + 65)
        elif 26 <= data.iloc[row,0] <= 35:
            data.loc[row, 'label'] = str(data.iloc[row,0] - 26)

In [ ]:
path = glob.glob('/content/runs/detect/predict/labels/*.txt')
path = [s.replace('\\','/') for s in path]
path = sorted(path, key = lambda x: int(x.split("/")[-1].split("DataVal")[1].split(".txt")[0]))

df = pd.DataFrame(columns = ['Name of File','License Plate'])

for file in path :
    colnames = ['class','xcenter','ycenter','width','height']
    data = pd.read_csv(file, names = colnames, delimiter = ' ')
    data.sort_values('xcenter', ascending = True, inplace = True)
    data.reset_index(inplace = True)
    data.drop(['index'], axis = 1, inplace = True)

    # Buat Kolom Label
    data_label(data)

    # Hasil predict
    hasil = list(data['label'])
    hasil = ''.join(hasil)

    start_pos = file.rfind("/") + 1
    end_pos = file.rfind(".")

    nama = file[start_pos:end_pos]
    nama = nama + ".png"

    # Masukin DataFrame
    df = df.append({'Name of File' : nama, 'License Plate' : hasil}, ignore_index = True)

In [ ]:
folder_path = "/content/gdrive/MyDrive/BDC 2023/Data Test"  # Ubah dengan path ke folder yang berisi file-file tersebut
file_names = os.listdir(folder_path)

data = pd.DataFrame(columns=['File', 'Prediksi'])
def filter_tab1(tab1):
    # Sort the DataFrame by 'xmin' in ascending order and 'confidence' in descending order
    tab1 = tab1.sort_values(by=['xmin', 'conf'], ascending=[True, False])

    # Initialize an empty DataFrame for the filtered rows
    filtered_rows = pd.DataFrame(columns=tab1.columns)

    # Iterate over the DataFrame and filter the rows
    xmin_threshold = 3
    prev_xmin = None
    max_confidence_row = None

    for _, row in tab1.iterrows():
        if prev_xmin is None or abs(row['xmin'] - prev_xmin) <= xmin_threshold:
            if max_confidence_row is None or row['conf'] > max_confidence_row['conf']:
                max_confidence_row = row
        else:
            if max_confidence_row is not None:
                filtered_rows = filtered_rows.append(max_confidence_row)

            max_confidence_row = row

        prev_xmin = row['xmin']

    if max_confidence_row is not None:
        filtered_rows = filtered_rows.append(max_confidence_row)

    # Reset the index of the filtered DataFrame
    filtered_rows = filtered_rows.reset_index(drop=True)

    return filtered_rows

for file_name in file_names:
    file_path = os.path.join(folder_path, file_name)

    # Lakukan proses pada setiap file
    names = model.names
    result = model.predict(source=file_path)
    result = result[0].cpu().numpy()

    class_names = []
    result_boxes = []

    for box in result.boxes:
        class_names.append(names[int(box.cls)])
        result_boxes.append(box.xyxy[0])

    pd_box = pd.DataFrame(result_boxes)
    pd_box.columns = ['xmin', 'ymin', 'xmax', 'ymax']
    pd_box['class'] = class_names
    pd_box['conf'] = result.boxes.conf
    pd_box = pd_box.sort_values('xmin')
    filtered_data = filter_tab1(pd_box)

    # Bikin DataFrame Hasil
    hasil = filtered_data['class'].str.cat(sep='')
    nama = os.path.splitext(file_name)[0] + ".png"

    # Buat final DataFrame
    data = data.append({'File': nama, 'Prediksi': hasil}, ignore_index=True)

# Tampilkan data frame hasil
print(data)